# Exercise 5.3

First of all, we import usefull libraries.

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from mlxtend.classifier import StackingClassifier

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Now, we import stored data.

In [2]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

We define a function to choose the best classifer.

In [3]:
def build_classifiers():
    
    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)
    
    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)
    
    svc = SVC()
    svc.fit(data_set, labels)
    
    decision_tree_classifier = DecisionTreeClassifier()
    decision_tree_classifier.fit(data_set, labels)
    
    gaussian_nb = GaussianNB()
    gaussian_nb.fit(data_set, labels)
    
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)
    
    return linear_regression, neighbors, svc

Then, we define another function to compare all of them.

In [4]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = QuadraticDiscriminantAnalysis()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

Finally, we build it and print their accuracy, so we're getting the best three ones.

In [5]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.85


/Users/Manuela/anaconda2/envs/py3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
